# Linguistic Analysis: Machine learning classifiers

In [ ]:
import pandas as pd
import os
import re
import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict
import math
import json
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from main_analyze import label_vocab
from main_analyze import get_main_data
df_all = get_main_data(assign_net_income_rank_flag=True)
df_all = df_all.query(" year<=2010 ") 
df_all = df_all.dropna(subset=['abstract_Marketing']).copy()
from main_analyze import label_vocab
interested_label_vocab = label_vocab[:6] 

In [ ]:
from sklearn.model_selection import train_test_split
X, Y = [], []
for idx, row in df_all.iterrows():
    text = row['abstract_Marketing']
    text = text.replace('-', ' ') 
    text = text.replace('_x000d_', ' ') 
    y = 1 if row['income_rank'] > 0.5 else 0
    X.append(text)
    Y.append(y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=1234)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42)),
    ])
text_clf.fit(X_train, Y_train)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
print(
    'accuracy_score',
    accuracy_score(Y_test, text_clf.predict(X_test))
)
auc = roc_auc_score(Y_test, text_clf.predict_proba(X_test)[:, 1])
print(
    'roc_auc_score',
    auc, 
    )
fpr, tpr, thresholds = roc_curve(Y_test, text_clf.predict_proba(X_test)[:, 1])
auc_tfidf = auc
fpr_tfidf, tpr_tfidf = fpr, tpr

In [ ]:
import seaborn as sns
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
sns.set(font_scale=1.4)  
X, Y = [], []
for idx, row in df_all.iterrows():
    category_labels = []
    for category_str in interested_label_vocab:
        category_labels.append(row[category_str])
    y = 1 if row['income_rank'] > 0.5 else 0
    X.append(category_labels)
    Y.append(y)
print('X', len(X), 'Y', len(Y), 'sum Y', sum(Y) )
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=1234)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
text_clf = Pipeline([
    ('clf', LogisticRegression(random_state=42)),
    ])
text_clf.fit(X_train, Y_train)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
print(
    'accuracy_score',
    accuracy_score(Y_test, text_clf.predict(X_test))
)
auc = roc_auc_score(Y_test, text_clf.predict_proba(X_test)[:, 1])
print(
    'roc_auc_score',
    auc
    )
fpr, tpr, thresholds = roc_curve(Y_test, text_clf.predict_proba(X_test)[:, 1])



plt.plot(fpr_BERT, tpr_BERT, label='%s ROC (area = %0.2f)' % ('BERT model', auc_BERT))
plt.plot(fpr_tfidf, tpr_tfidf, label='%s ROC (area = %0.2f)' % ('TF-IDF model', auc_tfidf))
plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('category baseline', auc))
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.savefig('notebook_plots/text_analysis_auc.jpg', dpi=300)
plt.show()
